In [52]:
import json
import torch

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

MODELS = [
    "Alibaba-NLP/gte-large-en-v1.5",
    "intfloat/multilingual-e5-large",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/all-MiniLM-L6-v2"
]

DATA_PATH = '../data/sentence-embeddings'
RELATIONS_JSON_PATH = '../data/relations.json'

In [3]:
with open(RELATIONS_JSON_PATH, 'r') as f:
    relations = json.load(f)['spatial_relations']

In [4]:
def get_relations_lookup(relations):
    relations_lookup = {}
    for category, category_pairs in relations.items():
        for first, second in category_pairs:
            relations_lookup[first] = {'category': category, 'opposite': second, 'position': 0}
            relations_lookup[second] = {'category': category, 'opposite': first, 'position': 1}
    return relations_lookup

relations_lookup = get_relations_lookup(relations)

In [39]:
def load_embeddings_for_model(model_name, relations_lookup):
    embeddings = []
    labels = []
    raw_data = torch.load(f'{DATA_PATH}/{model_name.replace("/", "_")}.pt', weights_only=False)
    for data_point in raw_data:
        embeddings.append(data_point['embedding'])
        labels.append(data_point['relation'])
    return np.array(embeddings), np.array(labels).reshape(-1, 1)

In [56]:
results = {}
models = {}
one_hot_encoder = OneHotEncoder()

for model_name in MODELS:
    print(f"Training probe for {model_name}...")
    X, y = load_embeddings_for_model(model_name, relations_lookup)
    y_encoded = one_hot_encoder.fit_transform(y).todense()
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2) 

    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    
    # Train probe
    clf = MLPClassifier(activation='identity')
    clf.fit(X_train, y_train)
    models[model_name] = clf

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    print(f"Accuracy for {model_name}: {accuracy:.2f}")

Training probe for Alibaba-NLP/gte-large-en-v1.5...
Accuracy for Alibaba-NLP/gte-large-en-v1.5: 1.00
Training probe for intfloat/multilingual-e5-large...
Accuracy for intfloat/multilingual-e5-large: 1.00
Training probe for sentence-transformers/all-mpnet-base-v2...
Accuracy for sentence-transformers/all-mpnet-base-v2: 1.00
Training probe for sentence-transformers/all-MiniLM-L6-v2...
Accuracy for sentence-transformers/all-MiniLM-L6-v2: 1.00


In [63]:
def load_embeddings_for_model_with_category_labels(model_name, relations_lookup):
    embeddings = []
    labels = []
    raw_data = torch.load(f'{DATA_PATH}/{model_name.replace("/", "_")}.pt', weights_only=False)
    for data_point in raw_data:
        embeddings.append(data_point['embedding'])
        relation = data_point['relation']
        category = relations_lookup[relation]['category']
        position = relations_lookup[relation]['position']
        labels.append(f'{category}-{position}')
    return np.array(embeddings), np.array(labels).reshape(-1, 1)

In [66]:
category_results = {}
category_models = {}
one_hot_encoder = OneHotEncoder()

for model_name in MODELS:
    print(f"Training probe for {model_name}...")
    X, y = load_embeddings_for_model_with_category_labels(model_name, relations_lookup)
    y_encoded = one_hot_encoder.fit_transform(y).todense()
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2) 

    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    
    # Train probe
    clf = MLPClassifier(activation='identity')
    clf.fit(X_train, y_train)
    category_models[model_name] = clf

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    category_results[model_name] = accuracy
    print(f"Accuracy for {model_name}: {accuracy:.2f}")

Training probe for Alibaba-NLP/gte-large-en-v1.5...
Accuracy for Alibaba-NLP/gte-large-en-v1.5: 1.00
Training probe for intfloat/multilingual-e5-large...
Accuracy for intfloat/multilingual-e5-large: 1.00
Training probe for sentence-transformers/all-mpnet-base-v2...
Accuracy for sentence-transformers/all-mpnet-base-v2: 1.00
Training probe for sentence-transformers/all-MiniLM-L6-v2...
Accuracy for sentence-transformers/all-MiniLM-L6-v2: 1.00
